In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback



import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'
projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]

In [3]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def tune_learner(learner, train_X, train_Y, tune_X, tune_Y, goal,loc=None,target_class=None):
    if not target_class:
        target_class = goal
    clf = learner(train_X, train_Y, tune_X, tune_Y, goal,loc)
    tuner = DE.DE_Tune_ML(clf, clf.get_param(), goal, target_class)
    return tuner.Tune()

In [4]:
class DE_Learners(object):
    def __init__(self, clf, train_X, train_Y, test_X, test_Y, goal,loc):
        """

        :param clf: classifier, SVM, etc...
        :param train_X: training data, independent variables.
        :param train_Y: training labels, dependent variables.
        :param predict_X: testing data, independent variables.
        :param predict_Y: testingd labels, dependent variables.
        :param goal: the objective of your tuning, F, recision,....
        """
        self.train_X = train_X
        self.train_Y = train_Y
        self.test_X = test_X
        self.test_Y = test_Y
        self.loc = loc
        self.goal = goal
        self.param_distribution = self.get_param()
        self.learner = clf
        self.confusion = None
        self.params = None
        
    def apply_smote(self,df,neighbours,r,up_to_num,auto):
        cols = df.columns
        smt = SMOTE.smote(df,neighbor = neighbours,r = r,up_to_num=up_to_num,auto=auto)
        df = smt.run()
        df.columns = cols
        return df

    def learn(self,F, **kwargs):
        """
        :param F: a dict, holds all scores, can be used during debugging
        :param kwargs: a dict, all the parameters need to set after tuning.
        :return: F, scores.
        """
        self.scores = {self.goal: [0.0]}
        try:   
            neighbours = kwargs.pop('neighbours')
            r = kwargs.pop('r')
            up_to_num = kwargs.pop('up_to_num')
            self.learner.set_params(**kwargs)
            _df = pd.concat([self.train_X, self.train_Y], axis = 1)
            _df = self.apply_smote(_df,neighbours,r,up_to_num,True)
            y_train = _df.Buggy
            X_train = _df.drop(labels = ['Buggy'],axis = 1)
            predict_result = []
            clf = self.learner.fit(X_train, y_train)
            predict_result = clf.predict(self.test_X)
            self.abcd = metrices.measures(self.test_Y,predict_result,self.loc)
            self.scores = self._Abcd(self.abcd,F)
            self.params = kwargs
        except Exception as e:
            #print(e)
            a = 10
        return self.scores
    
    def _Abcd(self,abcd , F):
        """

        :param predicted: predicted results(labels)
        :param actual: actual results(labels)
        :param F: previously got scores
        :return: updated scores.
        """
        if self.goal in ['f1','precision','recall','g-score','d2h','ifa','pci_20','pd','pf']:
            F['f1'] = [abcd.calculate_f1_score()]
            F['precision'] = [abcd.calculate_precision()]
            F['recall'] = [abcd.calculate_recall()]
            F['g-score'] = [abcd.get_g_score()]
            F['d2h'] = [abcd.calculate_d2h()]
            F['pci_20'] = [abcd.get_pci_20()]
            F['ifa'] = [abcd.get_ifa()]
            F['pd'] = [abcd.get_pd()]
            F['pf'] = [abcd.get_pf()]
            return F
        else:
            print("wronging goal")
            return F

    def predict(self,test_X):
        return self.learner.predict(test_X)

In [5]:
class SK_LR(DE_Learners):
    def __init__(self, train_x, train_y, predict_x, predict_y, goal,loc=None):
        clf = LogisticRegression()
        super(SK_LR, self).__init__(clf, train_x, train_y, predict_x, predict_y,goal,loc)

    def get_param(self):
        tunelst = {"penalty": ['l1', 'l2','elasticnet','none'],
                   "multi_class": ['ovr', 'multinomial','auto'],
                   "C": [1.0,200.0],
                   #"dual": [True, False],
                   #"fit_intercept": [True, False],
                   #"intercept_scaling": [1.0,100.0],
                   #"class_weight": ["balanced", None],
                   #"solver": ['newton-cg','lbfgs','liblinear','sag', 'saga'],
                   #"warm_start": [True, False],
                   "max_iter": [100,1000],
                   "neighbours": [5,21],
                   "r":[1,6],
                   "up_to_num": [1,4000]}
        return tunelst

In [6]:
final_score = {}
count = 0
for project in projects[0:1]:
    try:
        path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + project
        print(project)
        df = prepare_data(path)
        if count >= 10:
            break
        if df.shape[0] < 50:
            continue
        else:
            count+=1
        df.reset_index(drop=True,inplace=True)
        d = {'buggy': True, 'clean': False}
        df['Buggy'] = df['Buggy'].map(d)
        y = df.Buggy
        X = df.drop(labels = ['Buggy'],axis = 1)
        train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=0.33)
        df_train = pd.concat([train_X,train_y], axis = 1)
        df_test = pd.concat([test_X,test_y], axis = 1)
        df_train.reset_index(drop=True,inplace=True)
        y = df_train.Buggy
        X = df_train.drop(labels = ['Buggy'],axis = 1)
        kf = StratifiedKFold(n_splits = 5)
        goal = 'f1'
        learner = [SK_LR][0]
        F = {}
        score = {}
        for i in range(5):
            for train_index, tune_index in kf.split(X, y):
                X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
                y_train, y_tune = y[train_index], y[tune_index]
                _df = pd.concat([X_train,y_train], axis = 1)
                _df_tune = pd.concat([X_tune,y_tune], axis = 1)
                _df,selected_cols = apply_cfs(_df)
                y_train = _df.Buggy
                X_train = _df.drop(labels = ['Buggy'],axis = 1)
                _df_tune_loc = _df_tune.LOC
                _df_tune = _df_tune[selected_cols]
                y_tune = _df_tune.Buggy
                X_tune = _df_tune.drop(labels = ['Buggy'],axis = 1)
                df_test_loc = df_test.LOC
                _df_test = df_test[selected_cols]
                test_y = _df_test.Buggy
                test_X = _df_test.drop(labels = ['Buggy'],axis = 1)
                params, evaluation = tune_learner(learner, X_train, y_train,  X_tune,y_tune, goal,_df_tune_loc)
                clf = learner(X_train, y_train,  test_X,test_y, goal,df_test_loc)
                F = clf.learn(F,**params)
                _F = copy.deepcopy(F)
                if 'f1' not in score.keys():
                    score = _F
                else:
                    score['f1'].append(F['f1'][0])
                    score['precision'].append(F['precision'][0])
                    score['recall'].append(F['recall'][0])
                    score['g-score'].append(F['g-score'][0])
                    score['d2h'].append(F['d2h'][0])
                    score['pci_20'].append(F['pci_20'][0])
                    score['ifa'].append(F['ifa'][0])
                    score['pd'].append(F['pd'][0])
                    score['pf'].append(F['pf'][0])
            final_score[project] = score 
    except EnvironmentError as e:
        print(e)
        continue

bzbyte.csv
{'f1': 1.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.95}
{'f1': 0.94}
{'f1': 0.0}
{'f1': 1.0}
{'f1': 0.0}
{'f1': 0.89}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.94}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.94}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.94}
{'f1': 1.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.94}
{'f1': 1.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.89}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 1.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.94}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.95}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
TUNING DONE ! {'penalty': 'l2', 'multi_class': 'auto', 'C': 196.814, 'max_iter': 532, 'neighbours': 8, 'r': 3, 'up_to_num': 1264} {'f1': 1.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.93}
{'f1': 0.0}
{'f1': 0.0}
{'f1': 0.0}
{'f

KeyboardInterrupt: 

In [ ]:
np.median(final_score['bzbyte.csv']['f1'])

In [ ]:
with open('data/1385/1385_LR_DE_10.pkl', 'wb') as handle:
    pickle.dump(final_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pd.read_pickle('data/1385/1385_LR_DE_10.pkl')
results = []
for project in df.keys():
    results.append([project,np.median(df[project]['f1']),
                   np.median(df[project]['precision']),
                   np.median(df[project]['recall']),
                   np.median(df[project]['g-score']),
                   np.median(df[project]['d2h']),
                   np.median(df[project]['pci_20']),
                   np.median(df[project]['ifa']),
                   np.median(df[project]['pd']),
                   np.median(df[project]['pf'])])
results_df = pd.DataFrame(results,columns = ['project','f1','precision','recall','g-score','d2h','pci_20','ifa','pd','pf'])
results_df.to_csv('data/1385/1385_LR_DE_10.csv')

In [ ]:
project_score_df = pd.DataFrame(project_score, columns=['project','score'])

In [ ]:
project_score_df.to_csv('results_1385.csv')

In [ ]:
cnt = 0
for i in range(project_score_df.shape[0]):
    if round(np.median(np.array(project_score_df.iloc[i,1])),2) >= .75:
        print(project_score_df.iloc[i,0],round(np.median(np.array(project_score_df.iloc[i,1])),2))
        cnt+=1
print(cnt)
    #print(project_score_df.iloc[i,0],round(np.median(np.array(project_score_df.iloc[i,1])),2))

In [ ]:
df = prepare_data('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/coot.csv')

In [ ]:
count = 0
final_score = {}
for project in projects:
    try:
        path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + project
        print(project)
        df = prepare_data(path)
        df.reset_index(drop=True,inplace=True)
        d = {'buggy': True, 'clean': False}
        df['Buggy'] = df['Buggy'].map(d)
        df_0 = df[df['Buggy'] == True]
        df_1 = df[df['Buggy'] == False]
        final_score[project] = {'rows': df.shape[0], 'buggy': round(df_0.shape[0]/df.shape[0],2)}
    except Exception as e:
        print("error",e)
        continue

In [ ]:
final_score

In [ ]:
f1 = []
for i in range(project_score_df.shape[0]):
    print(project_score_df.iloc[i,0],np.median(project_score_df.iloc[i,1]))
    f1.append([project_score_df.iloc[i,0],np.median(project_score_df.iloc[i,1])])

In [ ]:
f1_df = pd.DataFrame(f1,columns = ['project','f1_median'])

In [ ]:
f1_df.to_csv('data/1385_plot.csv')

In [ ]:
final_score.keys()